# Chapter 5 線形独立

## 5.1 線形独立

## 5.2 基底

### キャッシュフローの複製

3期間のキャッシュフローを3次元ベクトルとして考えよう．本書＊＊ページでは以下のベクトルで基底を構成した．
$$
e_1 = 
\begin{bmatrix}
1\\
0\\
0
\end{bmatrix},
\quad
l_1 = 
\begin{bmatrix}
1\\
-(1+r)\\
0
\end{bmatrix},
\quad
l_2 = 
\begin{bmatrix}
0\\
1\\
-(1+r)
\end{bmatrix}
$$
ここで$r$は（正の）期間毎の金利である．
1つ目のベクトル$e_1$は期間$t=1$における1ドルの借り入れである．
2つ目のベクトル$l_1$は期間$t=1$における1ドルのローンで，期間$t=2$において金利$r$で支払う．
3つ目のベクトル$l_2$は期間$t=2$における1ドルのローンで，期間$t=3$において金利$r$で支払う．
この基底を用いてキャッシュフロー$c=(1,2,-3)$を以下のように複製しよう．
$$
c = \alpha_1 e_1 + \alpha_2 l_1 + \alpha_3 l_2 
= \alpha_1 
\begin{bmatrix}
1\\
0\\
0
\end{bmatrix} + 
\alpha_2 
\begin{bmatrix}
1\\
-(1+r)\\
0
\end{bmatrix} + 
\alpha_3 
\begin{bmatrix}
0\\
1\\
-(1+r)
\end{bmatrix}
$$

第3項から$c_3 = \alpha_3 (-(1+r))$となるので，$\alpha_3 = -c_3 / (1+r)$である．
第2項からは
$$
c_2 = \alpha_2 (-(1+r)) + \alpha_3 = \alpha_2 (-(1+r)) -c_3 / (1+r)
$$
となるので，$\alpha_2 = -c_2/(1+r) - c_3/(1+r)^2$である．
最後に，$c_1 = \alpha_1 + \alpha_2$であるので，
$$
\alpha_1 = c_1 + c_2/(1+r) + c_3/(1+r)^2
$$
となる．これはキャッシュフロー$c$の正味現在価値（NPV）である．

これをJuliaで確認してみよう．期間毎の金利は5%，キャッシュフローは$c=(1,2,-3)$とする．

In [2]:
r = 0.05;
e1 = [1, 0, 0];
l1 = [1, -(1 + r), 0];
l2 = [0, 1, -(1 + r)];
c = [1, 2, -3];
# 展開係数
alpha3 = -c[3] / (1 + r);
alpha2 = -c[2] / (1 + r) - c[3] / (1 + r)^2;
alpha1 = c[1] + c[2]/(1+r) + c[3]/(1+r)^2  # キャッシュフローのNPV

0.18367346938775508

In [3]:
alpha1 * e1 + alpha2 * l1 + alpha3 * l2

3-element Array{Float64,1}:
  1.0
  2.0
 -3.0

（後の章で，基底ベクトルの係数を自動的に求める簡単な方法を紹介する．）


## 5.3 直交ベクトル

### 正規直交基底による展開

以下のベクトル
$$
a_1 = 
\begin{bmatrix}
\phantom{-}0\\
\phantom{-}0\\
-1
\end{bmatrix},
\quad
a_2 = 
\frac{1}{\sqrt{2}}
\begin{bmatrix}
1\\
1\\
0
\end{bmatrix},
\quad
a_3 = 
\frac{1}{\sqrt{2}}
\begin{bmatrix}
\phantom{-}1\\
-1\\
\phantom{-}0
\end{bmatrix}
$$
が正規直交基底となることを確認しよう．またベクトル$x=(1,2,3)$のこの基底による展開
$$
x = (a_1^T x) a_1 + \cdots + (a_n^T x) a_n
$$
も確認しよう．

In [5]:
a1 = [0, 0, -1];
a2 = [1, 1, 0] / sqrt(2);
a3 = [1, -1, 0] / sqrt(2);

In [6]:
using LinearAlgebra

In [7]:
norm(a1), norm(a2), norm(a3)

(1.0, 0.9999999999999999, 0.9999999999999999)

In [8]:
a1' * a2, a1' * a3, a2' * a3

(0.0, 0.0, 0.0)

In [9]:
x = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

In [10]:
# 直交基底でのxの係数を求める
beta1 = a1' * x; 
beta2 = a2' * x; 
beta3 = a3' * x;
# xの基底での展開
xexp = beta1 * a1 + beta2 * a2 + beta3 * a3

3-element Array{Float64,1}:
 0.9999999999999999
 1.9999999999999996
 3.0               

## 5.4 グラム＝シュミット直交化

本書アルゴリズム5.1（グラム＝シュミットの直交化）をJuliaで実装しよう．引数として$k$個のベクトル$a_1, \ldots, a_k$からなる配列`[ a[1], a[2], ..., a[k] ]`をとる．これらのベクトルが線形独立であれば，グラム＝シュミット直交化で得られた正規直交ベクトル集合が配列`[ q[1], ..., q[k] ]`として返される．もしこれらのベクトルが線形従属であれば，グラム＝シュミット直交化は$i$回目の反復で終了し，長さ$i$の配列`[ q[1], ..., q[i] ]`を返す．

In [11]:
function gram_schmidt(a; tol = 1e-10)
    
q = []
for i = 1:length(a)
    qtilde = a[i]
    for j = 1:i-1
        qtilde -= (q[j]' * a[i]) * q[j]
    end
    if norm(qtilde) < tol
       println("Vectors are linearly dependent.")  # ベクトルは線形従属
       return q
    end
    push!(q, qtilde / norm(qtilde))
end;
return q
end

gram_schmidt (generic function with 1 method)

3行目では出力用配列を空配列で初期化している．各反復では`push!`関数で次のベクトルを配列に追加している（13行目）．

### 例

本書＊＊＊ページの例題にこの関数を適用してみる．

In [12]:
a = [ [-1, 1, -1, 1], [-1, 3, -1, 3], [1, 3, 5, 7] ]

3-element Array{Array{Int64,1},1}:
 [-1, 1, -1, 1]
 [-1, 3, -1, 3]
 [1, 3, 5, 7]  

In [13]:
q = gram_schmidt(a)

3-element Array{Any,1}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]  
 [-0.5, -0.5, 0.5, 0.5]

In [16]:
# 正規直交性の確認
norm(q[1])

1.0

In [17]:
q[1]' * q[2]

0.0

In [18]:
q[1]' * q[3]

0.0

In [19]:
norm(q[2])

1.0

In [20]:
q[2]' * q[3]

0.0

In [21]:
norm(q[3])

1.0

### 途中終了の例

ここで$a_3$を$a_1$と$a_2$の線形結合で置き換えると，入力するベクトル集合は線形従属になる．

In [22]:
b = [ a[1], a[2], 1.3 * a[1] + 0.5 * a[2] ]

3-element Array{Array{Float64,1},1}:
 [-1.0, 1.0, -1.0, 1.0]
 [-1.0, 3.0, -1.0, 3.0]
 [-1.8, 2.8, -1.8, 2.8]

In [23]:
q = gram_schmidt(b)

Vectors are linearly dependent.


2-element Array{Any,1}:
 [-0.5, 0.5, -0.5, 0.5]
 [0.5, 0.5, 0.5, 0.5]  

### 線型独立と次元の不等式の例

任意の3つの2次元ベクトルは線形従属である．グラム＝シュミット直交化を使って，これを確認しよう．

In [24]:
three_two_vectors = [ [1,1], [1,2], [-1,1] ];
q = gram_schmidt(three_two_vectors)

Vectors are linearly dependent.


2-element Array{Any,1}:
 [0.707107, 0.707107] 
 [-0.707107, 0.707107]